In [4]:
import sys
import os
from pathlib import Path

project_root = Path.cwd().parent 
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.predictive_models import (
    load_datasets,
    build_frequency_model,
    predict_frequency_model,
    build_sensationalism_model,
    predict_sensationalism_model,
    build_malicious_account_model,
    predict_malicious_account_model,
    build_naive_realism_model,
    predict_naive_realism_model,
)

import pandas as pd



# Ground Truth

In [10]:
ground_truth = pd.read_csv('../data/ground_truth.csv')
ground_truth.head()

,statement,frequency_heuristic,malicious_account,sensationalism,naive_realism,link
0,A president steeped in constitutional lore mig...,2,0,2,2,https://www.cnn.com/2025/10/20/politics/trump-...
1,"Trump is 'committed' to $2,000 tariff dividend...",0,0,0,1,https://abc7.com/post/trump-is-committed-givin...
2,The longest US government shutdown in history ...,0,0,0,1,https://www.theguardian.com/us-news/2025/nov/1...
3,Add AP News as your preferred source to see mo...,0,0,0,0,https://apnews.com/article/redistricting-gerry...
4,"Key Points Prop 50 passed in California, appro...",1,0,1,0,https://azcapitoltimes.com/news/2025/11/12/pro...


# Base Prompt

# Chain of Thought

# In Context Learning

In [16]:
icl_df = pd.read_csv('../webapp/cot_outputs.csv')
icl_df.head()

,id,url,freq_score,freq_reason,freq_confidence,mal_score,mal_reason,mal_confidence,sens_score,sens_reason,sens_confidence,naive_score,naive_reason,naive_confidence,overall_confidence
0,846eba04-9a07-4db2-8e2b-bd22b5e637ee,https://www.cnn.com/2025/10/20/politics/trump-...,2,The article heavily repeats the central narrat...,100,2,The article is a piece of speculative fiction ...,100,2,The article uses highly dramatic and exaggerat...,100,2,The article presents its critical perspective ...,100,100.00
1,fc5d28a2-d97d-4b17-97ad-897599641450,https://abc7.com/post/trump-is-committed-givin...,0,The article reports on a specific policy propo...,100,0,The sources cited are all credible and clearly...,100,1,While the article's own prose is neutral and o...,95,0,The article is a strong example of balanced re...,100,98.75
2,486dca50-6def-4ed3-b9eb-378771925237,https://www.theguardian.com/us-news/2025/nov/1...,0,The article reports on a singular event and it...,100,0,The article is a piece of standard political r...,100,1,While the headline is factual and the tone is ...,95,0,The article is well-balanced and explicitly pr...,100,98.75
3,adb23f07-5f8c-45a7-8bd6-8ebac3e55e91,https://apnews.com/article/redistricting-gerry...,0,The article introduces a central theme—mid-dec...,100,0,The article attributes information to credible...,100,1,The tone is largely informational and objectiv...,95,0,The article presents a nuanced situation by re...,90,96.25
4,c94d27e2-2d60-45ed-99f6-99edf4e4db0d,https://azcapitoltimes.com/news/2025/11/12/pro...,1,The article consistently repeats and reinforce...,95,0,The article is presented as a piece of politic...,100,0,The article maintains a neutral and analytical...,95,0,"The article is highly balanced and nuanced, ex...",100,97.50


In [17]:
icl_df

,id,url,freq_score,freq_reason,freq_confidence,mal_score,mal_reason,mal_confidence,sens_score,sens_reason,sens_confidence,naive_score,naive_reason,naive_confidence,overall_confidence
0,846eba04-9a07-4db2-8e2b-bd22b5e637ee,https://www.cnn.com/2025/10/20/politics/trump-...,2,The article heavily repeats the central narrat...,100,2,The article is a piece of speculative fiction ...,100,2,The article uses highly dramatic and exaggerat...,100,2,The article presents its critical perspective ...,100,100.00
1,fc5d28a2-d97d-4b17-97ad-897599641450,https://abc7.com/post/trump-is-committed-givin...,0,The article reports on a specific policy propo...,100,0,The sources cited are all credible and clearly...,100,1,While the article's own prose is neutral and o...,95,0,The article is a strong example of balanced re...,100,98.75
2,486dca50-6def-4ed3-b9eb-378771925237,https://www.theguardian.com/us-news/2025/nov/1...,0,The article reports on a singular event and it...,100,0,The article is a piece of standard political r...,100,1,While the headline is factual and the tone is ...,95,0,The article is well-balanced and explicitly pr...,100,98.75
3,adb23f07-5f8c-45a7-8bd6-8ebac3e55e91,https://apnews.com/article/redistricting-gerry...,0,The article introduces a central theme—mid-dec...,100,0,The article attributes information to credible...,100,1,The tone is largely informational and objectiv...,95,0,The article presents a nuanced situation by re...,90,96.25
4,c94d27e2-2d60-45ed-99f6-99edf4e4db0d,https://azcapitoltimes.com/news/2025/11/12/pro...,1,The article consistently repeats and reinforce...,95,0,The article is presented as a piece of politic...,100,0,The article maintains a neutral and analytical...,95,0,"The article is highly balanced and nuanced, ex...",100,97.50
5,f23bd343-374d-4442-bf09-1c42e7749dd0,https://abcnews.go.com/Politics/house-vote-ful...,0,The article reports on a sequence of events (a...,100,2,The article is highly deceptive as it contains...,100,0,The article's tone is intentionally neutral an...,95,2,The article is fully dogmatic because it prese...,95,97.50
6,2baed70e-45fe-4c96-b1d0-ea10e6bfaefd,https://www.nbcnews.com/politics/politics-news...,1,The article repeatedly references a specific p...,100,2,The article is a piece of disinformation based...,100,1,While the core subject matter—a political assa...,95,2,The article presents a completely fictional ev...,100,98.75
7,f0d709d6-255f-4e98-89b0-2631eb21e6bd,https://www.bbc.com/news/articles/c4gpnz05kr8o,1,The article reports on a central claim—that Ch...,95,0,"The article's source is the BBC, a credible ne...",100,1,The subject matter (international espionage) i...,90,0,The article is well-balanced and does not pres...,100,96.25
8,f3b6ddef-0a3a-4070-92e5-7e692ec692e0,https://calmatters.org/commentary/2025/11/dana...,0,The article reports on a specific event (a fed...,100,0,The article exhibits the characteristics of st...,100,1,The article uses mildly sensational language t...,95,0,"The article, while presenting a strong narrati...",95,97.50
9,238b7d86-bf71-49dd-90f5-afdbea9427ea,https://www.kff.org/racial-equity-and-health-p...,0,"The article reports findings from a single, cl...",100,0,The article is explicitly attributed to a part...,100,1,The article uses some emotionally charged lang...,95,0,The article is a report on survey findings and...,100,98.75


# Function Calling

# Fractal Chain of Thought